In [1]:
!pip install pyautogen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 336.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [2]:
import os
import json
from autogen import ConversableAgent

# put the fine-tune model to the LM studio, get the api key and base url , rewrite the config_list
llama3_config_list={"config_list":[{
    'model': 'meta/llama3-70b-instruct', #model here is your model name in the LM studio
    'api_key': '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',
    'base_url': "https://integrate.api.nvidia.com/v1",
    # 'api_version': '2024-02-15-preview',
    # 'api_type': 'azure',
}],
"cache_seed":None,
}


class ReportWriteReview():
    def __init__(self,report_keypoint):
        self.write_agent = ConversableAgent(
            "Jack (Analyst)",
            llm_config=llama3_config_list,
            system_message='''
            Your name is Jack, You are an equity research writer. Your task is to write a report on the company's financial performance and future prospects based on the provided company analysis brainstoming point information provided from your director. Your report should include a brief introduction of company, analysis of company's recent financial performance, how the recent event and news influence stock price and an analysis of the company's future prospects and risk. Your report should be written in English and be at least 300 words long. Also if receiving feedback from your director, you should be able to revise the report based on the feedback. The output should always follow the format: [Introduction of company]/n.../n/n[Analysis based on news and industry perspect]/n/n.../n/n[Analysis on Financial Report]/n/n.../n/n[Risk]/n/n.../n/n[Forecast]/n/n.../n/n[Recommendation]/n/n.../n
            '''
            )
        self.review_agent= ConversableAgent(

            "Emma(Director)",
            llm_config=llama3_config_list,
            system_message='''
            Your name is Emma, You are an professional equity research director. Your task is to review the report written by your analyst and provide feedback on the report. Your feedback should include detail suggestions for improvement, additional information that should be included, and any corrections that need to be made. Your feedback should be written in English and be at least 100 words long.
            '''
        )
        self.report_keypoint=report_keypoint

    def start_task(self):
        chat_result=self.review_agent.initiate_chat(self.write_agent,message="Jack, please write an equity research report for this company based on below information: "+self.report_keypoint,max_turn=3)
        return chat_result




if __name__=="__main__":
    report_keypoint='''[公司介绍]:\n\n腾讯控股是一家在软件服务行业的领先实体，自2004年06月16日成立并公开交易。截止今天，腾讯控股的总市值为3572736890364.0人民币，总股本数为9406890180.0，流通市值为3572736890364.0人民币，流通股数为9406890180.0。\n\n腾讯控股主要在中国运营，以股票代码00700在交易所进行交易。\n\n自2024-01-07至2024-01-14，腾讯控股的股票价格由288.80下跌至285.00，涨跌幅为：下跌1-2%。在此期间的公司新闻如下:\n\n[新闻标题]：腾讯控股00700.HK)连续33日回购 累计回购5978.00万股\n[新闻内容]：证券时报·数据宝统计，腾讯控股在港交所公告显示，1月8日以每股285.600港元至294.400港元的价格回购174.00\n\n如下所列为腾讯控股近期的一些金融基本面信息，记录时间为2023-12-31 00:00:00:\n\n[金融基本面]:\n\n报告期: 2023-12-31 00:00:00\n收入增长QOQ: 1.710497968345\n利润增长QOQ: 5.675512948548\n债务比例: 44.6071823926\n流水比例: 1.472201319298\n\n基于在2024-01-14之前的所有信息，让我们首先分析腾讯控股的积极发展和潜在担忧。请简洁地陈述，分别提出2-4个最重要的因素。大部分所提及的因素应该从公司的相关新闻中推断出来。那么让我们假设你对于下一周(2024-01-14至2024-01-21)的预测是下跌超过5%。提供一个总结分析来支持你的预测。预测结果需要从你最后的分析中推断出来，因此不作为你分析的基础因素。'''
    report_writer=ReportWriteReview(report_keypoint)
    chat_result=report_writer.start_task()
    print(chat_result)

Emma(Director) (to Jack (Analyst)):

Jack, please write an equity research report for this company based on below information: [公司介绍]:

腾讯控股是一家在软件服务行业的领先实体，自2004年06月16日成立并公开交易。截止今天，腾讯控股的总市值为3572736890364.0人民币，总股本数为9406890180.0，流通市值为3572736890364.0人民币，流通股数为9406890180.0。

腾讯控股主要在中国运营，以股票代码00700在交易所进行交易。

自2024-01-07至2024-01-14，腾讯控股的股票价格由288.80下跌至285.00，涨跌幅为：下跌1-2%。在此期间的公司新闻如下:

[新闻标题]：腾讯控股00700.HK)连续33日回购 累计回购5978.00万股
[新闻内容]：证券时报·数据宝统计，腾讯控股在港交所公告显示，1月8日以每股285.600港元至294.400港元的价格回购174.00

如下所列为腾讯控股近期的一些金融基本面信息，记录时间为2023-12-31 00:00:00:

[金融基本面]:

报告期: 2023-12-31 00:00:00
收入增长QOQ: 1.710497968345
利润增长QOQ: 5.675512948548
债务比例: 44.6071823926
流水比例: 1.472201319298

基于在2024-01-14之前的所有信息，让我们首先分析腾讯控股的积极发展和潜在担忧。请简洁地陈述，分别提出2-4个最重要的因素。大部分所提及的因素应该从公司的相关新闻中推断出来。那么让我们假设你对于下一周(2024-01-14至2024-01-21)的预测是下跌超过5%。提供一个总结分析来支持你的预测。预测结果需要从你最后的分析中推断出来，因此不作为你分析的基础因素。

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 06-20 13:38:54]

APIConnectionError: Connection error.